<a href="https://colab.research.google.com/github/elsa9421/Interactive-IPython-Demos/blob/main/Spectral_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Spectral Clustering 

The following notebook demonstrates Spectral Clustering using the python library sklearn.

Spectral Clustering treats each data point as a graph-node and thus transforms the clustering problem into a graph-partitioning problem. 


A typical implementation consists of three fundamental steps:-
1.  Build a Similarity Graph:-

    This step builds the Similarity Graph in the form of an adjacency matrix which is represented by A. The adjacency matrix can be built in the following manners:-
    -  **Epsilon-neighbourhood Graph**: A parameter `epsilon` is fixed beforehand. Then, each point is connected to all the points which lie in it’s epsilon-radius.  `Tunable Parameter` : `epsilon`

    -  **K-Nearest Neighbours**: A parameter k is fixed beforehand. Then, for two vertices u and v, an edge is directed from u to v only if v is among the k-nearest neighbours of u.   `Tunable Parameter` : `k`

    - **Fully-Connected Graph**: To build this graph, each point is connected with an undirected edge-weighted by the distance between the two points to every other point. Since this approach is used to model the local neighbourhood relationships thus typically the Gaussian similarity metric(i.e `Gaussian Kernel`) is used to calculate the distance.  `Tunable parameter` : `bandwidth`


2. Projecting the data onto a lower Dimensional Space:  Using Graph Laplacian Matrix
3. Clustering the Data: Typically, using K-Means clustering algorithm. 


* In this notebook we use python library function `sklearn.cluster.SpectralClustering` for steps 2 and 3







In [ ]:
from sklearn.datasets import make_moons,make_blobs,make_circles
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors
from math import log,exp
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from itertools import cycle, islice
sns.set()

In [ ]:

# Generate Data Points

X_circles,_= make_circles(n_samples=100, factor=.3, noise=.06,random_state=21)
X_moons,_ = make_moons(n_samples=100, noise=0.07 ,random_state=21)


def draw_graph(G,pos,title):
    '''
    Plots the Similarity Graph
    Input:
    G :- The Graph created by an adjacency matrix. It has information about number of nodes and how nodes are connected
         i.e G(V,E), where V= vertices (or nodes) and E- edges
    pos:- Position of indiviual nodes in form {0:(x0,y0),1:(x1,y1)...}
          where 0 is node number and (x0,y0) is the position of the node0
    '''
    ax = plt.gca()
    nx.draw_networkx_nodes(G, pos,node_size=70,ax=ax,label='Node')
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.6,label='edge')
    ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
    plt.title("Similarity Graph: {}".format(title),fontsize=14)
    plt.legend()



def plot_portfolio(graph_type='Epsilon Ball Graph , constant=1',epsilon=0.4,k=2,bandwidth=1,data="Half Moons"):

  '''
  The function plots :
  1. Similarity graph ( with datapoints and edges between closely related datapoints using the Adjacency matrix) 
  2. Graph which classifies datapoint using Spectral Clustering based on type of Similarity graph given    
    - epsilon ball graph      : Tunable parameter = epsilon     
    - k-nearest neighbor graph: Tunable parameter = k     
    - gaussian kernel  : Tunable parameter = bandwidth
  '''
  sns.set()
  #epsilon=round(exp(epsilon),3)
  if data=="Half Moons":
    X=X_moons
    
  if data== 'Concentric Circles':
    X=X_circles

  pos = {row:(X[row,0],X[row,1]) for row in range(len(X))}   # To obtain position of each node  in form {0:(x,y),1:(x,y)...}
                                                               # where 0 is node number and (x,y) is the position of the node0

  plt.figure(figsize=(16,8))
  plt.subplot(1,2,1)

  if graph_type=='Epsilon Ball Graph , constant=1':

    # Creating distance matrix with Euclidean' distance metric
    W = pairwise_distances(X, metric="euclidean")  # (N,N)-> where N is number of datapoints
    vectorizer = np.vectorize(lambda x: 1 if x < epsilon else 0)  
    W = np.vectorize(vectorizer)(W)  # to obtain adjacency matrix with weights=1 if distance <epsilon
    G=nx.from_numpy_matrix(W)        # generates a graph given the Adjacency matrix
    title='\u03B5={} ; Constant=1'.format(epsilon)

  
  if graph_type=='Epsilon Ball Graph , Gaussian':

    # Creating distance matrix with Euclidean' distance metric
    dist_matrix = pairwise_distances(X, metric="euclidean")  # (N,N)-> where N is number of datapoints

    ## Assigning weights based on Gaussian Similarity measure! Ensures weights are within 0 to 1
    W=np.exp(- dist_matrix ** 2 / (2. * bandwidth ** 2))   ## greater the similarity closer the value to 1
    W[dist_matrix<epsilon]=0                               ## Assigning zero weight to any distances which are within epsilon
    G=nx.from_numpy_matrix(W)        # generates a graph given the Adjacency matrix


    title='\u03B5={} ; Gaussian :BW={}'.format(epsilon,bandwidth)


  if graph_type=='k-Nearest Neighbors,constant=1':

    # Using predefined python function to obtain k nearest neighbors' graph
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    W=nbrs.kneighbors_graph(X).toarray()  #produce a sparse graph showing the connections between neighboring points and convert numpy Adjacency matrix

     # make connectivity symmetric
    W = 0.5 * (W + W.T)
    G=nx.from_numpy_matrix(W)        # generates a graph given the Adjacency matrix
    title='K={}-Nearest Neigbors ; Constant=1 '.format(k)

  
  if graph_type=='k-Nearest Neighbors, Gaussian':

    # Creating distance matrix with Euclidean' distance metric
    dist_matrix = pairwise_distances(X, metric="euclidean")  # (N,N)-> where N is number of datapoints
    W=np.exp(- dist_matrix ** 2 / (2. * bandwidth ** 2))

    # Using predefined python function to obtain k nearest neighbors' graph
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)

    #Computes the (weighted) graph of k-Neighbors for points in X
    W_nbrs=nbrs.kneighbors_graph(X).toarray()  #produce a sparse graph showing the connections between neighboring points and convert to numpy Adjacency matrix

     # make connectivity symmetric
    W_nbrs = 0.5 * (W_nbrs + W_nbrs.T)
    G=nx.from_numpy_matrix(W_nbrs)        # generates a graph given the Adjacency matrix
    title='K={}-Nearest Neigbors ; Gaussian: BW={}'.format(k,bandwidth)
    
    # Only consider weights for k neighbors!
    W[W_nbrs==0]=0



  if graph_type=='Fully Connected, Gaussian':

    dist_matrix=pairwise_distances(X, metric="euclidean")
    W=np.exp(- dist_matrix ** 2 / (2. * bandwidth ** 2))

    G=nx.from_numpy_matrix(W)        # generates a graph given the Adjacency matrix
    title='Fully Connected, BW={}'.format(bandwidth)

  
  draw_graph(G,pos,title)   

  
  sc = SpectralClustering(2, affinity='precomputed', n_init=100)
  sc=sc.fit(W)

  plt.subplot(1,2,2)
  plt.scatter(X[sc.labels_==0,0], X[sc.labels_==0,1], alpha=0.7, edgecolors='b',c='purple',s=50, label="Class 0")
  plt.scatter(X[sc.labels_==1,0], X[sc.labels_==1,1], alpha=0.7, edgecolors='b',c='yellow',s=50, label ="Class 1")
  plt.title("Classification after Spectral Clustering",fontsize=16)
  plt.legend()


    



  


data_widget=widgets.Dropdown(
                      options=['Half Moons', 'Concentric Circles'],
                      value='Half Moons',
                      disabled=False,
                      
                      )

data_w=widgets.HBox([widgets.Label('Dataset Selection    :'), data_widget])


graph_widget=widgets.Dropdown(
                      options=['Epsilon Ball Graph , constant=1','Epsilon Ball Graph , Gaussian','k-Nearest Neighbors,constant=1','k-Nearest Neighbors, Gaussian','Fully Connected, Gaussian'],
                      value='Epsilon Ball Graph , constant=1',
                      disabled=False,
                      )


graph_w=widgets.HBox([widgets.Label('Similarity Graph (Graph structure, Similarity Measure) :'), graph_widget])


k_slider=widgets.IntSlider(value=2,
                                 min=1,
                                 max=15,
                                 step=1,
                                # description='k:',
                                 continuous_update=False)



k_text=widgets.IntText(value=2,
                                 min=1,
                                 max=15,
                                 step=1,
                                 continuous_update=False)


widgets.link((k_slider, 'value'), (k_text, 'value'))
k_widget=widgets.HBox([widgets.Label('k:    :'),k_slider,k_text])





epsilon_slider=widgets.FloatSlider(value=0.4,
                                 min=0.06,
                                 max=2,
                                 step=.001,
                                 continuous_update=False)



epsilon_widget=widgets.HBox([widgets.Label('\u03B5:    :'),epsilon_slider])




epsilon_text=widgets.FloatText(value=0.4,
                                 min=0.06,
                                 max=2,
                                 step=.001,
                                 continuous_update=False)


widgets.link((epsilon_slider, 'value'), (epsilon_text, 'value'))
epsilon_widget=widgets.HBox([widgets.Label('\u03B5:    :'),epsilon_slider,epsilon_text])


# epsilon_widget=widgets.HBox([epsilon_slider,epsilon_text])




bandwidth_slider=widgets.FloatSlider(value=0.05,
                                 min=0.002,
                                 max=2,
                                 step=.05,
                                 continuous_update=False)



bandwidth_text=widgets.FloatText(value=0.05,
                                 min=0.002,
                                 max=2,
                                 step=.05,
                                 continuous_update=False)





widgets.link((bandwidth_slider, 'value'), (bandwidth_text, 'value'))
bandwidth_widget=widgets.HBox([widgets.Label('Bandwidth   :'),bandwidth_slider,widgets.Label('Bandwidth    :'),bandwidth_text])

Menu=widgets.HBox([data_w,graph_w])


# data_widget=widgets.Dropdown(
#                       options=['Half Moons', 'Concentric Circles'],
#                       value='Half Moons',
#                       disabled=False,
                      
#                       )

# graph_widget=widgets.Dropdown(
#                       options=['Epsilon Ball Graph , constant=1','Epsilon Ball Graph , Gaussian','k-Nearest Neighbors,constant=1','k-Nearest Neighbors, Gaussian','Fully Connected, Gaussian'],
#                       value='Epsilon Ball Graph , constant=1',
#                       disabled=False,
#                       )


def update_default_values(*args):
  '''
  To update epsilon, k and bandwidth to the optimal parameters 

  '''
  if data_widget.value=='Half Moons' and graph_widget.value=='Epsilon Ball Graph , constant=1':
    epsilon_slider.value=0.4
  elif data_widget.value=='Half Moons' and graph_widget.value=='Epsilon Ball Graph , Gaussian':
    epsilon_slider.value=0.2
    bandwidth_text.value=0.10
  elif data_widget.value=='Half Moons' and graph_widget.value=='k-Nearest Neighbors,constant=1':
    k_slider.value=11
  elif data_widget.value=='Half Moons' and graph_widget.value=='k-Nearest Neighbors, Gaussian':
    k_slider.value=6
    bandwidth_slider.value=0.15
  elif data_widget.value=='Half Moons' and graph_widget.value=='Fully Connected, Gaussian':
    bandwidth_slider.value=0.15

  if data_widget.value=='Concentric Circles' and graph_widget.value=='Epsilon Ball Graph , constant=1':
    epsilon_slider.value=0.30
  elif data_widget.value=='Concentric Circles' and graph_widget.value=='Epsilon Ball Graph , Gaussian':
    epsilon_slider.value=0.18
    bandwidth_text.value=0.15
  elif data_widget.value=='Concentric Circles' and graph_widget.value=='k-Nearest Neighbors,constant=1':
    k_slider.value=5
  elif data_widget.value=='Concentric Circles' and graph_widget.value=='k-Nearest Neighbors, Gaussian':
    k_slider.value=4
    bandwidth_slider.value=0.05
  elif data_widget.value=='Concentric Circles' and graph_widget.value=='Fully Connected, Gaussian':
    bandwidth_slider.value=0.20




data_widget.observe(update_default_values,'value')
graph_widget.observe(update_default_values,'value')







out=widgets.interactive_output(plot_portfolio,{"epsilon":epsilon_text,"graph_type":graph_widget,"k":k_text,"bandwidth":bandwidth_text,"data":data_widget})
%time display(Menu,epsilon_widget,k_widget,bandwidth_widget,out)



Output()

CPU times: user 24.5 ms, sys: 380 µs, total: 24.8 ms
Wall time: 48.4 ms
